In [ ]:
# doc2vec_model : https://drive.google.com/open?id=1XnK7GETFweWKW22UQj9SgePFRYgSg9DF

In [26]:
import codecs
from gensim.models import Doc2Vec
from konlpy.tag import Twitter
from scipy.spatial.distance import cosine

In [96]:
model = Doc2Vec.load('./doc2vec_model/doc2vec_twitter_kowiki_300000_docs.model')

In [69]:
model.wv.most_similar('경제', topn=20)

[('경제정책', 0.41632378101348877),
 ('재정', 0.39564135670661926),
 ('정치', 0.35518383979797363),
 ('관광업', 0.33051955699920654),
 ('조선업', 0.3285074830055237),
 ('흘러감', 0.3268545866012573),
 ('산업', 0.32522839307785034),
 ('경공업', 0.31397348642349243),
 ('루청', 0.308829665184021),
 ('경제력', 0.30664265155792236),
 ('무역', 0.3056638836860657),
 ('칠국', 0.3049558401107788),
 ('판매업', 0.3044031262397766),
 ('GDP', 0.2997964918613434),
 ('중화학공업', 0.29525649547576904),
 ('시장경제', 0.2929162085056305),
 ('Trickle', 0.29010969400405884),
 ('구매력', 0.28562217950820923),
 ('상업', 0.28235161304473877),
 ('GRDP', 0.2823108434677124)]

In [4]:
model.wv.most_similar(positive=['대한민국', '도쿄'], negative=['서울'])

[('일본', 0.40066996216773987),
 ('일본국', 0.3375188410282135),
 ('도쿄도', 0.3237033486366272),
 ('방수로', 0.2989196479320526),
 ('이시하라', 0.29582974314689636),
 ('안보투쟁', 0.29407060146331787),
 ('스이킨쿠쓰', 0.2803184986114502),
 ('류코쿠', 0.2792055904865265),
 ('Scorpions', 0.2721102237701416),
 ('도치기현', 0.2690941393375397)]

In [12]:
with codecs.open('./train_kin/train_data.txt', encoding='utf-8') as f:
    data = f.readlines()
    data = list(map(lambda x: x.strip().split('\t'), data))
    
with open('./train_kin/train_label.txt') as f:
    label = f.readlines()
    label = list(map(lambda x: int(x), label))

In [15]:
twitter = Twitter()

In [19]:
def tagger(sent):
    return [word for word, tag in twitter.pos(sent) if tag not in ['Punctuation', 'Unknown']]

In [41]:
def inference(model):
    def proc(sent):
        return model.infer_vector(filter(lambda x: x in model.wv.vocab, tagger(sent)))
    
    return proc

infer = inference(model)

In [13]:
data[0], label[0]

(['경제가 불안하면 국채를 왜발행하나요??', '국채 발행 이유?'], 1)

In [59]:
list(filter(lambda x: x in model.wv.vocab, tagger(data[0][1])))

['국채', '발행', '이유']

In [61]:
cosine(*map(infer, data[0]))

1.027425640391644

In [63]:
model.wv.cosine_similarities(infer(data[0][0]), infer(data[0][1]).reshape(1, -1))

array([-0.02742564], dtype=float32)

In [73]:
for (d1, d2), l in zip(data, label):
    print('{} : {} : "{}", "{}"'.format(l, 1 - cosine(infer(d1), infer(d2)), d1, d2))

1 : -0.02742564039164397 : "경제가 불안하면 국채를 왜발행하나요??", "국채 발행 이유?"
0 : 0.09183199996440328 : "경제가 불안하면 국채를 왜발행하나요??", "국채를 많이 발행하면 왜 수요가 떨어져요?"
0 : 0.02353202162673962 : "경제가 불안하면 국채를 왜발행하나요??", "국채발행은 왜 필요한가"
0 : -0.05448374687739177 : "경제가 불안하면 국채를 왜발행하나요??", "국채를 발행하며 왜 시장 금리가 올라가나요?"
0 : -0.04242315752786241 : "경제가 불안하면 국채를 왜발행하나요??", "나라가 국채를 발행할때"
0 : 0.009169175859680578 : "국채 발행 이유?", "국채를 많이 발행하면 왜 수요가 떨어져요?"
0 : 0.004363854076830043 : "국채 발행 이유?", "국채발행은 왜 필요한가"
0 : 0.05393645817045656 : "국채 발행 이유?", "국채를 발행하며 왜 시장 금리가 올라가나요?"
0 : -0.033140409658841685 : "국채 발행 이유?", "나라가 국채를 발행할때"
1 : 0.01972162332097216 : "외환보유고 에 관한 질문", "외환보유고에 대해..."
1 : -0.05851407407165787 : "외환보유고 에 관한 질문", "한국의 외환보유고에 관한 질문"
1 : 0.04731059111916225 : "외환보유고 에 관한 질문", "외환보유고에 대해서"
1 : 0.008360609618123349 : "외환보유고 에 관한 질문", "한국의 외환 보유고 (추가내공 有)"
1 : -0.03987869119726195 : "외환보유고에 대해...", "한국의 외환보유고에 관한 질문"
1 : -0.03224660310873895 : "외환보유고에 대해...", "외환보유고에 대해서"
1 : -0.13035075795190965 : "외환보유고에 대해...", 

In [74]:
cos = list(map(lambda x: 1 - cosine(*map(infer, x)), data))

In [84]:
from sklearn.linear_model import LogisticRegression
import numpy as np

In [85]:
lr = LogisticRegression()

In [86]:
cos[0], label[0]

(-0.02742564039164397, 1)

In [87]:
lr.fit(np.reshape(cos, (-1, 1)), label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [91]:
lr.score(np.reshape(cos, (-1, 1)), label)

0.6697247706422018

In [92]:
import tensorflow as tf

C:\Users\revsi\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [100]:
def vectorize(model):
    def vec(sent):
        return model.wv[filter(lambda x: x in model.wv.vocab, tagger(sent))]
    
    return vec

vec = vectorize(model)